In [ ]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime
import pytz
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium
#Update protobuf
!pip install --upgrade protobuf

# Installer Pyroute
!pip install pyroutelib3
from pyroutelib3 import Router

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


# Connection API

In [ ]:
# IMPORTER LES DONNEES DE L'API VELOS DE TOULOUSE ET LES STOCKER DANS UN DF

lien_information = "https://transport.data.gouv.fr/gbfs/toulouse/station_information.json"
lien_status = "https://transport.data.gouv.fr/gbfs/toulouse/station_status.json"

# Charger le fichier JSON en tant que DataFrame
df_info = pd.read_json(lien_information)
df_statut = pd.read_json(lien_status)

# Stations & disponibilités

In [ ]:
# Affichage de l'ensemble des données de la colonne Data pour récupérer les stations
stations_list = df_info['data'][0]

# Convertir la liste de dictionnaires en DataFrame
df_stations = pd.DataFrame(stations_list)

# Affichage de l'ensemble des données de la colonne Data pour récupérer les disponibilités
statut_list = df_statut['data'][0]

# Convertir la liste de dictionnaires en DataFrame
df_disponibilites = pd.DataFrame(statut_list)

# Convertir la colonne last_reported en objets datetime au fuseau horaire UTC
df_disponibilites['last_reported'] = pd.to_datetime(df_disponibilites['last_reported'], unit='s').dt.tz_localize('UTC')

# Convertir la colonne last_reported en heure locale Paris
paris_tz = pytz.timezone('Europe/Paris')
df_disponibilites['last_reported'] = df_disponibilites['last_reported'].dt.tz_convert(paris_tz)

# Formater la colonne last_reported en une chaîne de caractères sans indication de fuseau horaire
df_disponibilites['last_reported'] = df_disponibilites['last_reported'].dt.strftime('%d/%m/%Y %H:%M:%S')

# Merge des deux DF stations et disponibilités
df_velo_temps_reel = pd.merge(df_stations,
        df_disponibilites,
        how="left",
        left_on='station_id',
        right_on='station_id')

# Supprimer le code station de la colonne 'name' => démarrer au 8ème caractère
df_velo_temps_reel['name'] = df_velo_temps_reel['name'].apply(lambda x : x[8:])

# Créer une colonne adresse complete pour avoir l'adresse et le nom de la ville (utile pour la suite)
df_velo_temps_reel['adresse_complete'] = df_velo_temps_reel['address']+", , Toulouse"

# concatener la latitude avec la longitude
df_velo_temps_reel['lat&lon'] = df_velo_temps_reel.apply(lambda x: f"{x['lat']}, {x['lon']}", axis=1)

# df_velo_temps_reel

# Station vélo la plus proche 

In [ ]:
# AFFICHER LA STATION DE VELO LA PLUS PROCHE DU PARKING CHOISI

# EN AMONT NOUS AVONS RECUPERER L'ADRESSE DU PARKING CHOISI

# POUR LE RESTE
adresse_du_parking_choisi = 'Place du Capitole, 31000 Toulouse'
# Adresse de référence
address_reference = adresse_du_parking_choisi

# Liste des coordonnées à comparer
coordinates_to_compare = df_velo_temps_reel['lat&lon']

# Initialisation du géocodeur
geolocator = Nominatim(user_agent="my_app")

# Géocodage de l'adresse de référence
location_reference = geolocator.geocode(address_reference)

# Vérification des coordonnées géographiques
if location_reference is not None:

    # Extraction des coordonnées de l'adresse de référence
    coordinates_reference = (location_reference.latitude, location_reference.longitude)

    # Initialisation des variables pour le calcul de la distance minimale
    min_distance = float('inf')
    closest_address = None

    # Parcours des adresses à comparer
    for coordinate_to_compare in coordinates_to_compare:
        distance = geodesic(coordinates_reference, coordinate_to_compare).meters

        # Vérification si la distance est plus petite que la distance minimale actuelle
        if distance < min_distance:
            min_distance = distance
            closest_address = coordinate_to_compare

    if closest_address is not None:
        # Affichage des éléments suivants :
        list_1 = []
        list_2 = []
        list_3 = []
        list_4 = []
        list_5 = []
        print("Station vélo la plus proche: ", df_velo_temps_reel['name'][df_velo_temps_reel['lat&lon'] == (closest_address)].item())
        Station_velo_la_plus_proche = df_velo_temps_reel['name'][df_velo_temps_reel['lat&lon'] == (closest_address)].item()
        list_1.append(Station_velo_la_plus_proche)
        print("Nombre de vélos disponibles: ", df_velo_temps_reel['num_bikes_available'][df_velo_temps_reel['lat&lon'] == (closest_address)].item())
        nb_velos_dispos = df_velo_temps_reel['num_bikes_available'][df_velo_temps_reel['lat&lon'] == (closest_address)].item()
        list_2.append(nb_velos_dispos)
        print("Nombre de bornes disponibles: ", df_velo_temps_reel['num_docks_available'][df_velo_temps_reel['lat&lon'] == (closest_address)].item())
        nb_bornes_dispos = df_velo_temps_reel['num_docks_available'][df_velo_temps_reel['lat&lon'] == (closest_address)].item()
        list_3.append(nb_bornes_dispos)
        print("Adresse : ", df_velo_temps_reel['address'][df_velo_temps_reel['lat&lon'] == (closest_address)].item())
        adresse_de_la_station = df_velo_temps_reel['address'][df_velo_temps_reel['lat&lon'] == (closest_address)].item()
        list_4.append(adresse_de_la_station)
        print("Distance : {:.0f} mètres".format(min_distance))
        list_5.append(min_distance)


    else:
        print("Aucune adresse trouvée parmi la liste")
else:
    print("Adresse de référence introuvable")

df_station_velo_plus_proche = pd.concat([pd.Series(list_1), pd.Series(list_2), pd.Series(list_3), pd.Series(list_4),pd.Series(list_5)], axis=1)
df_station_velo_plus_proche = df_station_velo_plus_proche.rename(columns = {0 : 'Nom station', 1 : 'Nombre de vélos disponibles', 2 : 'Nombre de bornes disponibles', 3 : 'Adresse' , 4 : 'Distance'})
df_station_velo_plus_proche['Distance'] = df_station_velo_plus_proche['Distance'].astype(int)

Station vélo la plus proche:  LOIS
Nombre de vélos disponibles:  3
Nombre de bornes disponibles:  15
Adresse :  10 RUE DES LOIS
Distance : 119 mètres


In [ ]:
# DF A RECUPERER POUR AFFICHER LES ELEMENTS SUR DASH 
df_station_velo_plus_proche

,Nom station,Nombre de vélos disponibles,Nombre de bornes disponibles,Adresse,Distance
0,LOIS,3,15,10 RUE DES LOIS,118


# CARTE

In [ ]:
# AFFICHER LA STATION LA PLUS PROCHE ET LE PARCOURS POUR ALLER A LA STATION LA PLUS PROCHE DE SA DESTINATION

# Création de l'objet Router
router = Router("cycle")

# EN AMONT NOUS AVONS RECUPERE LE PARKING CHOISI : ici pour l'exemple
nom_du_parking_choisi = 'Capitole'

# Sélection des noms de départ et d'arrivée (exemple)
depart_name = nom_du_parking_choisi
arrivee_name = Station_velo_la_plus_proche

# A AFFICHER DANS DASH 

# Rechercher les lignes correspondant aux noms de départ et d'arrivée dans le DataFrame (df)
#depart_row = df_parking_global[df_parking_global['Parkings'] == depart_name].iloc[0]
arrivee_row = df_velo_temps_reel[df_velo_temps_reel['name'] == arrivee_name].iloc[0]

# Récupérer les coordonnées latitude/longitude du point de départ :
#depart_lat = depart_row['ylat']
#depart_lon = depart_row['xlong']

# POUR L'EXEMPLE ici et ne pas être obligé de recharger tout le DF parking global sur l'autre notebook
depart_lat = 43.6042
depart_lon = 1.44348

# Récupérer les coordonnées latitude/longitude du point d'arrivée
arrivee_lat = arrivee_row['lat']
arrivee_lon = arrivee_row['lon']

# Rechercher les nœuds de départ et d'arrivée
depart = router.findNode(depart_lat, depart_lon)
arrivee = router.findNode(arrivee_lat, arrivee_lon)

# Calculer l'itinéraire
status, itineraire = router.doRoute(depart, arrivee)

if status == 'success':
    # Convertir les coordonnées des nœuds en latitude/longitude
    itineraire_coordonnees = [router.nodeLatLon(node) for node in itineraire]

    # Créer la carte avec le point de départ, le point d'arrivée et l'itinéraire
    carte = folium.Map(location=[depart_lat, depart_lon], zoom_start=13)

    # Ajouter le marqueur pour le point de départ
    folium.Marker([depart_lat, depart_lon], popup="Point de départ").add_to(carte)

    # Ajouter le marqueur pour le point d'arrivée
    folium.Marker([arrivee_lat, arrivee_lon], popup="Point d'arrivée").add_to(carte)

    # Ajouter le tracé de l'itinéraire à la carte
    folium.PolyLine(
        locations=itineraire_coordonnees,
        color="blue",
        weight=2.5,
        opacity=1
    ).add_to(carte)

    # Afficher la carte dans la cellule Colab
    carte
else:
    print("Impossible de trouver un itinéraire pour les points spécifiés.")


In [ ]:
carte

# ANALYSE STATION AU FIL DES HEURES ET DES JOURS

In [ ]:
# Chargement du DF récupéré sur plusieurs jours 
df_analyse = pd.read_csv('df_analyse.csv')

# création de la colonne 'heure'
df_analyse['last_reported'] = pd.to_datetime(df_analyse['last_reported'])  # Conversion en objet datetime
df_analyse['heure'] = df_analyse['last_reported'].dt.time  # Extraction des heures et minutes

# modification de l'ordre des colonnes
new_order = ['station_id', 'name', 'address', 'num_bikes_available', 'num_docks_available', 'capacity', 'heure','lat', 'lon', 'is_installed', 'is_renting', 'is_returning', 'last_reported']
df_analyse = df_analyse.reindex(columns=new_order)

# Supprimer les lignes en doublons
#df_analyse = df_analyse.drop_duplicates() 

# Filtrer uniquement les lignes de la stations choisie
df_analyse_station_velo_choisie = df_analyse[df_analyse['name'] == nom_station_velo_choisie]

#Mise en conformité Datetime des valeurs dates du df avec création colonne hour et jour:
df_analyse_station_velo_choisie['jour_et_heure'] = pd.to_datetime(df_analyse_station_velo_choisie['last_reported'])
df_analyse_station_velo_choisie['hour'] = df_analyse_station_velo_choisie['jour_et_heure'].apply(lambda x: x.hour)
df_analyse_station_velo_choisie['jour'] = df_analyse_station_velo_choisie['jour_et_heure'].apply(lambda x: x.day)

#Création dico pour visibilité nom du jour et création colonne nom_jour :
jours = {'16': 'Vendredi 16 Juin','17': 'Samedi 17 Juin','19': 'Lundi 19 Juin','20': 'Mardi 20 Juin'}
df_analyse_station_velo_choisie['nom_jour'] = df_analyse_station_velo_choisie['jour'].apply(lambda x: jours[str(x)])


/tmp/ipykernel_2490/1244965339.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2490/1244965339.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2490/1244965339.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2

In [ ]:
fig = px.bar(df_analyse_station_velo_choisie, x='heure', y='num_bikes_available',animation_frame="nom_jour",range_x=[0,70],range_y=[0,25])

fig.update_layout(
        title=f"Nombre de vélos disponibles à la borne {Station_velo_la_plus_proche}",
        xaxis_title='Heure',
        yaxis_title='Nombre de vélos disponibles',
        #plot_bgcolor='#a5282b',  # Couleur de fond du graphique
        barmode='stack'  # Empilement des barres
    )

fig.update_traces(marker_color='#ffc12b')  # Attribution de la couleur aux barres

fig.update_xaxes(tickangle=45, tickmode='linear', tick0=0, dtick=2)  # Rotation des étiquettes des heures et affichage d'une ligne sur deux

fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7c80b052-7ead-4799-b132-a25d57707958' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>